In [1]:
import sys
sys.path.append('../..')
from src import WindSpeedUnit, ForecastAPI, ForecastMinutely15
import src
import datetime as dt

f = ForecastAPI()
f.latitude, f.longitude = src.GEO_COORDINATES.DUNDALK_IT.value
f.wind_speed_unit = WindSpeedUnit.METERS_PER_SECOND
f.minutely_15 = [ForecastMinutely15.WIND_SPEED_80M, ForecastMinutely15.WIND_DIRECTION_80M, ForecastMinutely15.TEMPERATURE_2M]
f.end_minutely_15 = dt.datetime.now() + dt.timedelta(days=10)
f.start_minutely_15 = dt.datetime.now()

response = f.request()

Remaining: {'daily': 9275.8, 'hourly': 4744.0, 'minutely': 600}
API counts: 1.0


In [2]:
import pandas as pd

df = pd.DataFrame(response['minutely_15'])
df.rename(columns = {'time':'Timestamps','wind_speed_80m':'WindSpeed','wind_direction_80m':'WindDirAbs','temperature_2m':'EnvirTemp'}, inplace = True)
df = df[['Timestamps','WindSpeed','WindDirAbs','EnvirTemp']]

df.Timestamps = df.Timestamps.astype('str')
df.WindSpeed = df.WindSpeed.astype('float')
df.WindDirAbs = df.WindDirAbs.astype('float')
df.EnvirTemp = df.EnvirTemp.astype('float')

df

,Timestamps,WindSpeed,WindDirAbs,EnvirTemp
0,2025-02-22T22:29,9.84,180.0,6.5
1,2025-02-22T22:44,10.14,181.0,6.6
2,2025-02-22T22:59,10.23,182.0,6.7
3,2025-02-22T23:14,10.04,180.0,6.7
4,2025-02-22T23:29,9.55,177.0,6.7
...,...,...,...,...
956,2025-03-04T21:29,14.10,246.0,7.4
957,2025-03-04T21:44,14.19,246.0,7.4
958,2025-03-04T21:59,14.19,246.0,7.4
959,2025-03-04T22:14,14.29,246.0,7.4


In [6]:
from src.daemon import ForecastDaemon

daemon = ForecastDaemon()

In [7]:
daemon.start()

Remaining: {'daily': 9274.8, 'hourly': 4743.0, 'minutely': 599.0}
API counts: 1.0
Remaining: {'daily': 9273.8, 'hourly': 4742.0, 'minutely': 598.0}
API counts: 1.0
Remaining: {'daily': 9272.8, 'hourly': 4741.0, 'minutely': 597.0}
API counts: 1.0
Remaining: {'daily': 9271.8, 'hourly': 4740.0, 'minutely': 596.0}
API counts: 1.0
Remaining: {'daily': 9270.8, 'hourly': 4739.0, 'minutely': 595.0}
API counts: 1.0
Remaining: {'daily': 9269.8, 'hourly': 4738.0, 'minutely': 594.0}
API counts: 1.0
Remaining: {'daily': 9268.8, 'hourly': 4737.0, 'minutely': 593.0}
API counts: 1.0


In [9]:
daemon.stop()

In [80]:
from src.weather.utils import ApiCounter, RequestLogger
ApiCounter.calculate_call_weight_from_url(f.build_url())
requestRemaining = RequestLogger.queryRemaining(f._conn)

In [90]:
def secondsTillEndOf(t:str):
    if t == 'minutely':
        return 60 - dt.datetime.now().second
    if t == 'hourly':
        return 3600 - dt.datetime.now().minute*60 - dt.datetime.now().second
    if t == 'daily':
        return 86400 - dt.datetime.now().hour*3600 - dt.datetime.now().minute*60 - dt.datetime.now().second
    else:
        raise ValueError('Invalid time unit. Must be minutely, hourly or daily')

secondsRemaining = {k:secondsTillEndOf(k) for k in ['minutely','hourly','daily']}
requestRemaining = {'daily': 9531.8, 'hourly': 5000, 'minutely': 600}

sleepTimes = {k:secondsRemaining[k]/requestRemaining[k] for k in ['minutely','hourly','daily']}
sleepTime = max(sleepTimes.values())
sleepTime

0.7562055435489625

In [10]:
pd.read_sql('SELECT * FROM real_time_predictions', f._conn)


,Timestamps,WindSpeed,WindDirAbs,EnvirTemp,PowerPrediction
0,2025-02-22T22:29,9.84,180.0,6.5,506.204742
1,2025-02-22T22:44,10.14,181.0,6.6,542.271851
2,2025-02-22T22:59,10.23,182.0,6.7,553.066406
3,2025-02-22T23:14,10.04,180.0,6.7,530.003479
4,2025-02-22T23:29,9.55,177.0,6.7,470.738129
...,...,...,...,...,...
956,2025-03-04T21:29,14.10,246.0,7.4,762.623840
957,2025-03-04T21:44,14.19,246.0,7.4,762.085876
958,2025-03-04T21:59,14.19,246.0,7.4,762.085876
959,2025-03-04T22:14,14.29,246.0,7.4,761.488098


331

59